In [10]:
import os
import requests
from bs4 import BeautifulSoup

# Функция для создания папки, если она не существует
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Функция для скачивания изображения
def download_image(url, folder_path):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            image_name = url.split("/")[-1]
            image_path = os.path.join(folder_path, image_name)
            with open(image_path, 'wb') as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            return image_path
    except Exception as e:
        print(f"Не удалось скачать изображение {url}. Ошибка: {e}")
        return None

# Функция для извлечения и скачивания изображений с HTML страницы
def extract_and_download_images(html_content, folder_path):
    soup = BeautifulSoup(html_content, 'html.parser')
    img_tags = soup.find_all('img')
    
    full_image_urls = []
    downloaded_images = []

    for img in img_tags:
        img_url = img.get('src')
        if img_url and img_url.startswith('http'):
            full_image_urls.append(img_url)
            image_path = download_image(img_url, folder_path)
            if image_path:
                downloaded_images.append(image_path)

    return full_image_urls, downloaded_images


In [11]:
# Основная часть скрипта
html_file_path = 'htmls/q1.html'
download_folder = 'htmls/q1'

# Создаем папку для сохранения изображений
create_directory(download_folder)

# Читаем HTML файл
with open(html_file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

# Извлекаем и скачиваем изображения
full_image_urls, downloaded_images = extract_and_download_images(html_content, download_folder)

# Сохраняем пути к полноразмерным изображениям в файл
with open('full_image_urls.txt', 'w', encoding='utf-8') as file:
    for url in full_image_urls:
        file.write(url + '\n')

print("Скачивание завершено. Пути к изображениям сохранены в full_image_urls.txt.")